In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Concatenate, Dense, Dropout, Softmax
from keras.callbacks import TensorBoard
from time import time

Using TensorFlow backend.


In [2]:
with open('./xtrain_obfuscated.txt') as fp:
    data = fp.read().split('\n')
with open('./ytrain.txt') as fp:
    label = fp.read().split('\n')

In [3]:
max_char = 26
max_len = 0
for sent in data:
    max_len = max(max_len, len(sent))

In [4]:
for i in range(len(label)):
    label[i] = int(label[i])
label = np.asarray(label)
ohe = OneHotEncoder(sparse=False)
label = label.reshape(-1,1)
y_train_data = ohe.fit_transform(label)

In [5]:
X_train = []
for i in range(len(data)):
    temp = np.zeros((max_len, max_char))
    for j in range(len(data[i])):
        temp[j][ord(data[i][j])-97] = 1
    X_train.append(temp)
X_train = np.asarray(X_train)
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1], X_train.shape[2],1))

In [6]:
convSize_1 = 3
convSize_2 = 4
convSize_3 = 5
convSize_4 = 6
convSize_5 = 7
convSize_6 = 8
convSize_7 = 9
convSize_8 = 10

inputLayer = Input(shape=(X_train[0].shape[0],X_train[0].shape[1],1))
convLayer_1 = Conv2D(filters=256, kernel_size=(convSize_1,X_train[0].shape[1]))(inputLayer)
convLayer_2 = Conv2D(filters=256, kernel_size=(convSize_2,X_train[0].shape[1]))(inputLayer)
convLayer_3 = Conv2D(filters=256, kernel_size=(convSize_3,X_train[0].shape[1]))(inputLayer)
convLayer_4 = Conv2D(filters=256, kernel_size=(convSize_4,X_train[0].shape[1]))(inputLayer)
convLayer_5 = Conv2D(filters=256, kernel_size=(convSize_5,X_train[0].shape[1]))(inputLayer)
convLayer_6 = Conv2D(filters=256, kernel_size=(convSize_6,X_train[0].shape[1]))(inputLayer)
convLayer_7 = Conv2D(filters=256, kernel_size=(convSize_7,X_train[0].shape[1]))(inputLayer)
convLayer_8 = Conv2D(filters=256, kernel_size=(convSize_8,X_train[0].shape[1]))(inputLayer)

dropout_1 = Dropout(0.5)(convLayer_1)
dropout_2 = Dropout(0.5)(convLayer_2)
dropout_3 = Dropout(0.5)(convLayer_3)
dropout_4 = Dropout(0.5)(convLayer_4)
dropout_5 = Dropout(0.5)(convLayer_5)
dropout_6 = Dropout(0.5)(convLayer_6)
dropout_7 = Dropout(0.5)(convLayer_7)
dropout_8 = Dropout(0.5)(convLayer_8)

maxPool_1 = MaxPooling2D(pool_size=(max_len-convSize_1+1, 1))(dropout_1)
maxPool_2 = MaxPooling2D(pool_size=(max_len-convSize_2+1, 1))(dropout_2)
maxPool_3 = MaxPooling2D(pool_size=(max_len-convSize_3+1, 1))(dropout_3)
maxPool_4 = MaxPooling2D(pool_size=(max_len-convSize_4+1, 1))(dropout_4)
maxPool_5 = MaxPooling2D(pool_size=(max_len-convSize_5+1, 1))(dropout_5)
maxPool_6 = MaxPooling2D(pool_size=(max_len-convSize_6+1, 1))(dropout_6)
maxPool_7 = MaxPooling2D(pool_size=(max_len-convSize_7+1, 1))(dropout_7)
maxPool_8 = MaxPooling2D(pool_size=(max_len-convSize_8+1, 1))(dropout_8)

flatten_1 = Flatten()(maxPool_1)
flatten_2 = Flatten()(maxPool_2)
flatten_3 = Flatten()(maxPool_3)
flatten_4 = Flatten()(maxPool_4)
flatten_5 = Flatten()(maxPool_5)
flatten_6 = Flatten()(maxPool_6)
flatten_7 = Flatten()(maxPool_7)
flatten_8 = Flatten()(maxPool_8)

mergedLayer = Concatenate(axis=1)([flatten_1, flatten_2, flatten_3, flatten_4, flatten_5, flatten_6, flatten_7, flatten_8])
dense_1 = Dense(1024, activation='relu')(mergedLayer)
dropout_9 = Dropout(0.5)(dense_1)
dense_2 = Dense(128, activation='relu')(dropout_9)
dropout_10 = Dropout(0.5)(dense_2)

result = Dense(12, activation='softmax')(dropout_10)

In [7]:
model = Model(inputLayer, result)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 452, 26, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 450, 1, 256)  20224       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 449, 1, 256)  26880       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 448, 1, 256)  33536       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir='./logs/{}'.format(time()))

In [10]:
model.fit(X_train, y_train_data, 
        batch_size=512, 
        verbose=1, epochs=20, 
        shuffle=True, 
        validation_split=0.2,
        callbacks=[tensorboard])

Train on 26010 samples, validate on 6503 samples
Epoch 1/20
26010/26010 [==============================] - 58s 2ms/step - loss: 2.3577 - acc: 0.1577 - val_loss: 2.2197 - val_acc: 0.2843
Epoch 2/20
26010/26010 [==============================] - 53s 2ms/step - loss: 1.7494 - acc: 0.3982 - val_loss: 1.3848 - val_acc: 0.5647
Epoch 3/20
26010/26010 [==============================] - 52s 2ms/step - loss: 1.2516 - acc: 0.5706 - val_loss: 1.1087 - val_acc: 0.6488
Epoch 4/20
26010/26010 [==============================] - 52s 2ms/step - loss: 1.0095 - acc: 0.6611 - val_loss: 0.9242 - val_acc: 0.7264
Epoch 5/20
26010/26010 [==============================] - 52s 2ms/step - loss: 0.8535 - acc: 0.7151 - val_loss: 0.8302 - val_acc: 0.7598
Epoch 6/20
26010/26010 [==============================] - 52s 2ms/step - loss: 0.7611 - acc: 0.7498 - val_loss: 0.7642 - val_acc: 0.7789
Epoch 7/20
26010/26010 [==============================] - 52s 2ms/step - loss: 0.6681 - acc: 0.7804 - val_loss: 0.6857 - val_acc:

ResourceExhaustedError: OOM when allocating tensor with shape[512,256,450,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/dropout_1/cond/Merge_grad/cond_grad"], data_format="NCHW", ksize=[1, 1, 450, 1], padding="VALID", strides=[1, 1, 450, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](dropout_1/cond/Merge, max_pooling2d_1/MaxPool, training/Adam/gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad-2-TransposeNHWCToNCHW-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [11]:
model.save('8_layer_conv.h5')